In [1]:
import sqlite3
import pandas as pd

# Данные

Представленный датасет включает многочисленные финансовые показатели, которые часто используются многими профессионалами и экономистами-аналитиками в области инвестиций для оценки компаний. Указанные компании входят в индекс S&P 500 (Standard & Poor's 500). S&P 500 - это взвешенный по капитализации индекс 500-сот крупнейших публичных компаний США.
https://www.kaggle.com/paytonfisher/sp-500-companies-with-financial-information

- 3 переменные символьного типа:
        - Symbol: Символ тикера, используемый для однозначной идентификации каждой компании на определенном фондовом рынке.
        - Name: Юридическое название компании
        - Sector: Отрасль экономики, к которой относятся выпускаемые продукты и/или услуги компании.
   
- 8 переменных числового типа:
        - Price: Цена акции
        - Price to Earnings (PE): Отношение цены акции компании к ее прибыли на акцию.
        - Dividend Yield: Отношение годовых дивидендов на акцию к цене акции.
        - Earnings Per Share (EPS): отношение чистой прибыли компании, доступной для распределения, к среднегодовому числу обыкновенных акций
        - 52 week high and low: Годовой максимум и минимум цены акции компании
        - Market Cap: Рыночная стоимость акций компании (рассчитывается как цена акции x количество акций)
        - EBITDA: Прибыль компании до вычета процентов, налогов, износа и амортизации; часто используется как показатель прибыльности

In [2]:
data_companies = pd.read_csv('dataset_1.csv')
data_financial_metrics = pd.read_csv('dataset_2.csv')

In [3]:
data_companies.head()

,id,Symbol,Name,Sector,Price
0,1,ABT,Abbott Laboratories,Health Care,56.27
1,2,ALGN,Align Technology,Health Care,220.71
2,3,AMZN,Amazon.com Inc,Consumer Discretionary,1350.50
3,4,ILMN,Illumina Inc,Health Care,209.54
4,5,TMO,Thermo Fisher Scientific,Health Care,198.73


In [4]:
data_financial_metrics.head()

,id,Symbol,PriceEarnings,DividendYield,EarningsShare,X52WeekLow,X52WeekHigh,MarketCap,EBITDA
0,1,ABT,22.51,1.908982,0.26,64.60,42.2800,1.021210e+11,5.744000e+09
1,2,ALGN,56.59,0.000000,2.84,287.32,92.6100,1.878804e+10,3.803260e+08
2,3,AMZN,296.16,0.000000,6.16,1498.00,812.5000,6.858734e+11,1.613200e+10
3,4,ILMN,52.25,0.000000,4.92,248.97,158.0203,3.229520e+10,1.192000e+09
4,5,TMO,21.84,0.327632,5.60,226.44,151.6900,8.322659e+10,4.751300e+09


# Создание базы данных

In [6]:
connection = sqlite3.connect('S&P500.db')
connection.execute("PRAGMA foreign_keys = ON")

## Создание и загрузка данных в таблицы

In [7]:
connection.execute('''CREATE TABLE IF NOT EXISTS companies (
id INTEGER PRIMARY KEY,
Symbol TEXT,
Name TEXT,
Sector TEXT,
Price FLOAT
)''')

In [8]:
connection.execute('''CREATE TABLE IF NOT EXISTS financial_metrics (
id INTEGER PRIMARY KEY,
Symbol TEXT,
PriceEarnings FLOAT,
DividendYield FLOAT,
EarningsShare FLOAT,
X52WeekLow FLOAT,
X52WeekHigh FLOAT,
MarketCap FLOAT,
EBITDA FLOAT,
FOREIGN KEY (id) REFERENCES companies(id)
ON UPDATE CASCADE
ON DELETE CASCADE
)''')

In [9]:
df = pd.read_sql_query("SELECT * FROM financial_metrics", connection)
df.head()

,id,Symbol,PriceEarnings,DividendYield,EarningsShare,X52WeekLow,X52WeekHigh,MarketCap,EBITDA


In [10]:
# Загрузка данных
data_companies.to_sql('companies', connection, if_exists='append', index=False)
data_financial_metrics.to_sql('financial_metrics', connection, if_exists='append', index=False)

# Команды: UPDATE, DELETE И CASCADE

### UPDATE

In [11]:
connection.execute('UPDATE companies SET Price=60 WHERE Symbol="ABT"')
connection.execute('SELECT * FROM companies WHERE Symbol="ABT" ').fetchone()

(1, 'ABT', 'Abbott Laboratories', 'Health Care', 60.0)

### DELETE

In [12]:
connection.execute('SELECT * FROM companies').fetchmany(5)

[(1, 'ABT', 'Abbott Laboratories', 'Health Care', 60.0),
 (2, 'ALGN', 'Align Technology', 'Health Care', 220.71),
 (3, 'AMZN', 'Amazon.com Inc', 'Consumer Discretionary', 1350.5),
 (4, 'ILMN', 'Illumina Inc', 'Health Care', 209.54),
 (5, 'TMO', 'Thermo Fisher Scientific', 'Health Care', 198.73)]

In [13]:
connection.execute('DELETE FROM companies WHERE Price < 100')
connection.execute('SELECT * FROM companies WHERE Price < 100 ').fetchmany(5)

[]

### CASCADE

In [14]:
connection.execute('DELETE from companies WHERE id < 5')
connection.execute('SELECT * FROM companies''').fetchmany(5)

[(5, 'TMO', 'Thermo Fisher Scientific', 'Health Care', 198.73),
 (6, 'V', 'Visa Inc.', 'Information Technology', 113.86),
 (7, 'AON', 'Aon plc', 'Financials', 136.05),
 (11, 'BLK', 'BlackRock', 'Financials', 509.38),
 (17, 'MCK', 'McKesson Corp.', 'Health Care', 150.23)]

In [15]:
connection.execute('SELECT * FROM financial_metrics''').fetchall()

[(5,
  'TMO',
  21.84,
  0.32763190000000003,
  5.6,
  226.44,
  151.69,
  83226586345.0,
  4751300000.0),
 (6,
  'V',
  32.72,
  0.70204765,
  2.8,
  126.88,
  84.88,
  270038723213.0,
  13086000000.0),
 (7,
  'AON',
  20.8,
  1.0245464,
  4.66,
  152.78,
  113.2201,
  35123123422.0,
  1858000000.0),
 (11,
  'BLK',
  22.49,
  2.1643557999999996,
  30.3,
  594.52,
  368.0,
  85907759858.0,
  5684000000.0),
 (17,
  'MCK',
  11.68,
  0.8898775999999999,
  22.74,
  178.86,
  133.82,
  31534840262.0,
  7232000000.0),
 (18,
  'AGN',
  10.65,
  1.643289,
  38.35,
  256.8,
  160.07,
  56668833898.0,
  -2888100000.0),
 (19, 'NFLX', 200.08, 0.0, 1.25, 286.81, 138.26, 114805404842.0, 809028000.0),
 (22,
  'ECL',
  28.08,
  1.2319711000000002,
  4.14,
  140.5,
  119.61,
  38460272282.0,
  2848600000.0),
 (23, 'CI', 18.11, 0.020466639, 7.2, 227.13, 141.93, 47680910480.0, 0.0),
 (24, 'ARE', 19.03, 3.0262272, 1.57, 134.37, 106.89, 12043374429.0, 0.0),
 (26,
  'PPG',
  18.73,
  1.5771488999999999,
  